In [14]:
!pip install --upgrade transformers trl datasets
!pip install --upgrade unsloth


In [1]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')


In [2]:
import torch
from unsloth import FastVisionModel
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
    ignore_mismatched_sizes=True  # Add this line
)

==((====))==  Unsloth 2024.11.11: Fast Mllama vision patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/385k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [4]:
# Defining layer which we are going to fine-tune
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers = True,
    finetune_attention_modules =True,
    finetune_mlp_modules = True,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

In [5]:
# Load the dataset, combining train and validation splits for a larger training set
dataset = load_dataset("unsloth/Radiology_mini", split="train")

instruction = """You are an expert radiographer.
Carefully examine the provided medical image.
Describe your observations accurately and comprehensively, including any abnormalities or significant features.
Based on your expertise, suggest the next steps or potential solutions, if applicable.
Ensure your response is clear, concise, and professional."""

README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/481M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/79.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1978 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/327 [00:00<?, ? examples/s]

In [6]:
def convert_to_conversation(sample):
  conversation = [
      {
          "role": "user",
          "content": [
              {
                  "type": "text",
                  "text": instruction

              },
              {
                  "type": "image",
                  "image": sample["image"]

              }
          ]
      },
      {
          "role": "assistant",
          "content": [
              { "type": "text", "text" : sample["caption"]}
          ]
      }
  ]

  return { "messages" : conversation }

In [7]:
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

In [8]:
FastVisionModel.for_inference(model)
image = dataset[5]["image"]
instruction = """You are an expert radiographer.
Carefully examine the provided medical image.
Describe your observations accurately and comprehensively, including any abnormalities or significant features.
Based on your expertise, suggest the next steps or potential solutions, if applicable.
Ensure your response is clear, concise, and professional."""

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",

            },
            {
                "type": "text",
                "text": instruction
            }
        ]
    }
]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    return_tensors="pt",
    add_special_tokens=False
).to("cuda")

print("\nBT-Answer:\n")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=125,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)


BT-Answer:

The provided medical image appears to be an angiogram, specifically an image of the subclavian artery. The subclavian artery is a major blood vessel that runs through the neck and shoulders.

**Observations:**

*   **Normal Subclavian Artery:** The subclavian artery is depicted in its typical location, supplying blood to the arms.
*   **Bulge in the Subclavian Artery:** A notable abnormality is observed, characterized by a bulge or dilation in the subclavian artery, proximal to its branching points into the axillary and vertebral arteries. This suggests a


In [9]:
FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer),
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 30,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048
    )
)

max_steps is given, it will override any value given in num_train_epochs


In [11]:

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer_stats = trainer.train()

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = Tesla T4. Max memory = 14.748 GB.
10.16 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,978 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 67,174,400
🦥 Unsloth needs about 1-3 minutes to load everything - please wait!


Step,Training Loss
1,0.959300
2,0.859700
3,1.117200
4,0.820100
5,0.905400
6,0.788900
7,0.946800
8,0.846500
9,0.807600
10,0.687500


1004.4017 seconds used for training.
16.74 minutes used for training.
Peak reserved memory = 10.338 GB.
Peak reserved memory for training = 0.178 GB.
Peak reserved memory % of max memory = 70.098 %.
Peak reserved memory for training % of max memory = 1.207 %.


In [17]:

from PIL import Image


print("\nAfter training:\n")
FastVisionModel.for_inference(model)
image = Image.open("/content/tessst.jpg")

instruction =  """You are an expert radiographer.
Carefully examine the provided medical image, but more precised please.
Describe your observations accurately and comprehensively, including any abnormalities or significant features.
Based on your expertise, suggest the next steps or potential solutions, if applicable.
Ensure your response is clear, concise, and professional."""

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 600,
                   use_cache = True, temperature = 1.5, min_p = 0.1)



After training:

T1 weighted MRI scan following intravenous gadolinium administration, demonstrating complete resolution of all cystic changes and enhancement.<|eot_id|>


In [9]:
# 6. Save the model

#model.save_pretrained("BS_MedX_MedChat_weights_2")
#tokenizer.save_pretrained("BS_MedX_MedChat_tokenizer_2")
model.push_to_hub("BS_MedX_MedChat_tokenizer_2", tokenizer)

#model.save_pretrained_merged("BS_MedX_MedChat", tokenizer)


NameError: name 'model' is not defined

In [ ]:
#model.push_to_hub(
 #  "BSAtlas/BS_MedX_MedChat",
  # tokenizer,
   #save_method="merged_16bit",
   #token=HF_TOKEN
#)

Flask App


In [ ]:
# !pip install flask flask-ngrok transformers torch

In [ ]:
!pip install streamlit -q
!pip install pyngrok
# # #!npm install -g localtunnel


In [ ]:
# !pip install streamlit requests

In [ ]:
# %%writefile streamlit_app.py
# import streamlit as st
# from PIL import Image
# import requests
# import json

# def main():
#     st.title("MedChat: Your Medical Assistant")
#     st.write("Upload an image and provide an instruction for evaluation.")

#     # Image uploader
#     image_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

#     # Instruction input
#     instruction = st.text_input("Enter instruction:")

#     if image_file is not None and instruction:
#         # Open and display the uploaded image
#         image = Image.open(image_file)
#         st.image(image, caption="Uploaded Image", use_container_width=True)

#         # Placeholder for actual image processing logic
#         image_input = "Image preprocessed"

#         # Add a loading spinner while the model is generating the response
#         with st.spinner("Generating response... Please wait."):
#             # Send request to Flask API
#             response = requests.post(
#                 'http://<ngrok-id>.ngrok.io/generate',  # Replace with your ngrok URL
#                 json={'instruction': instruction, 'image_input': image_input}
#             )
#             response_data = response.json()
#             model_response = response_data['response']

#         # Display response after generation is done
#         st.subheader("Model Response:")
#         st.write(model_response)
#     else:
#         st.write("Please upload an image and enter an instruction.")

# if __name__ == "__main__":
#     main()

In [ ]:
# %%writefile run_apps.sh
# #!/bin/bash

# # Start the Flask app in the background
# nohup python flask_app.py &

# # Wait for a few seconds to ensure the Flask app starts
# sleep 5
# python gnok_app.py
# # Run the Streamlit app
# streamlit run streamlit_app.py --server.port 8502 &

In [ ]:
%cd /content/
print("\nlist of content folder")
%ls
#print("\nlist of BS_MedX_MedChat_weights folder\n")
%ls /content/BS_MedX_MedChat_weights/
print("\nlist of BS_MedX_MedChat_tokenizer folder\n")
%ls /content/BS_MedX_MedChat_tokenizer/
#print("\nlist of BS_MedX_MedChat_merged folder\n")
#%ls /content/BS_MedX_MedChat/

/content

list of content folder
app.py                        BS_MedX_MedChat_weights_2/  outputs/      streamlit_app.py
BS_MedX_MedChat_tokenizer/    flask_app.py                __pycache__/  testtttttt.jpg
BS_MedX_MedChat_tokenizer_2/  gnok_app.py                 run_apps.sh*  unsloth_compiled_cache/
BS_MedX_MedChat_weights/      nohup.out                   sample_data/
adapter_config.json  adapter_model.safetensors  README.md

list of BS_MedX_MedChat_tokenizer folder

chat_template.json        special_tokens_map.json  tokenizer.json
preprocessor_config.json  tokenizer_config.json


In [19]:
# #!pip install --upgrade torch
!pip install --upgrade streamlit transformers torch pyngrok
#!pip install tensorflow
!pip install nvidia-tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-10.6.0.post1-py2.py3-none-any.whl size=16416 sha256=dccce044ca622e2760723381b01546b8bf4229064d593293a81ace8e2a50546d
  Stored in directory: /root/.cache/pip/wheels/d6/d4/72/a641e4695bb1b8b57fa2ad1235493c73c469a080b16e5375a9
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.6.0.post1-py2.py3-none-any.whl size=17631 sha256=e96bc8395dc663a2381b1733d07f271fe786a045217a77409412067665e97bc9
  Stored in directory: /root/.cache/pip/wheels/aa/cb/33/33a230ed5c2a54da56fd68d4edd3adff93b82812124fbf9efd
Successfully built tensorrt tensorrt-cu12


In [20]:

from PIL import Image


#print("\nAfter training:\n")
image = Image.open("/content/tessst.jpg")

def generate_text_from_image(model, tokenizer, image, instruction):
    # Preprocess the image here (you can extend this with actual image preprocessing logic)
    #image_input = image  # Placeholder for actual image processing logic

    # Concatenate instruction and image description

    prompt = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
    prompt = tokenizer.apply_chat_template(prompt, add_generation_prompt = True)
    # Move model and inputs to the appropriate device
    device = "cuda"
    model.to(device)  # Ensure the model is on GPU

    # Tokenize the input prompt and move tensors to GPU
    inputs = tokenizer(
    image,
    prompt,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")


    # Check for invalid values in the input tensors
    if torch.isnan(inputs.input_ids).any() or torch.isinf(inputs.input_ids).any():
        raise ValueError("Input tensor contains NaN or Inf values.")

    # Use mixed precision to reduce memory usage
    from torch.amp import autocast
    with autocast('cuda'):
        # Use TextStreamer for efficient text generation
        text_streamer = TextStreamer(tokenizer, skip_prompt=True)
        outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=600, use_cache=True, temperature=1.5, min_p=0.1)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
print(generate_text_from_image(model, tokenizer, image, instruction))

Magnetic resonance imaging (MRI) in T1-weighted sequences demonstrating the complete disappearance of the cerebral haematoma in all three dimensions, with slight cerebral oedema and subdural collection<|eot_id|>
user

You are an expert radiographer.
Carefully examine the provided medical image, but more precised please.
Describe your observations accurately and comprehensively, including any abnormalities or significant features.
Based on your expertise, suggest the next steps or potential solutions, if applicable.
Ensure your response is clear, concise, and professional.assistant

Magnetic resonance imaging (MRI) in T1-weighted sequences demonstrating the complete disappearance of the cerebral haematoma in all three dimensions, with slight cerebral oedema and subdural collection


In [30]:
%%writefile app.py
import streamlit as st
from unsloth import FastVisionModel
from PIL import Image
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import os
from transformers import BitsAndBytesConfig

# Set environment variables for CUDA debugging and optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['TORCH_USE_CUDA_DSA'] = '1'  # Enable device-side assertions
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Enable detailed CUDA error messages

# Load model and tokenizer
@st.cache_resource
def load_model():
    # Load quantization config
    quant_config = BitsAndBytesConfig(load_in_4bit=True)
    # Load the model with quantization
    model = AutoModelForCausalLM.from_pretrained(
        "/content/BS_MedX_MedChat_weights_2",
        quantization_config=quant_config
    )

    # Load tokenizer
    _, tokenizer = FastVisionModel.from_pretrained(
        "unsloth/Llama-3.2-11B-Vision-Instruct",
        load_in_4bit = True,
        use_gradient_checkpointing="unsloth",
        ignore_mismatched_sizes=True  # Handle potential size mismatches
    )

    # Move the model to GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    return model, tokenizer

# Generate text from image and instruction
def generate_text_from_image(model, tokenizer, image, instruction):
    # Prepare the prompt
    prompt = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]
    prompt = tokenizer.apply_chat_template(prompt, add_generation_prompt=True)

    # Move image and prompt to the correct device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Tokenize inputs
    inputs = tokenizer(
        image,
        prompt,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)

    # Check and clean inputs
    inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}

    # Use mixed precision for memory efficiency
    from torch.cuda.amp import autocast
    with autocast(device_type='cuda'):
        # Initialize text streamer
        text_streamer = TextStreamer(tokenizer, skip_prompt=True)
        # Generate response
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            streamer=text_streamer,
            max_new_tokens=600,
            use_cache=True,
            temperature=1.5,
            min_p=0.1
        )

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Streamlit app interface
def main():
    st.title("MedChat: Your Medical Assistant")
    st.write("Upload an image and provide an instruction for evaluation.")

    # Image uploader
    image_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

    # Instruction input
    instruction = st.text_input("Enter instruction:")

    if image_file is not None and instruction:
        # Load model and tokenizer
        model, tokenizer = load_model()

        # Open and display the uploaded image
        image = Image.open(image_file)
        st.image(image, caption="Uploaded Image", use_container_width=True)

        # Add a loading spinner while generating response
        with st.spinner("Generating response... Please wait."):
            try:
                # Generate response
                response = generate_text_from_image(model, tokenizer, image, instruction)
                st.subheader("Model Response:")
                st.write(response)
            except ValueError as e:
                st.error(f"An error occurred: {e}")
    else:
        st.write("Please upload an image and enter an instruction.")

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
# # import torch
# # print(torch.cuda.is_available())  # Should print True if GPU is available
# # print(torch.cuda.current_device())  # Show the current GPU device ID
# # print(torch.cuda.get_device_name(0))  # Check the name of the GPU
# import os
# # os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# # !nvidia-smi

# os.environ['TORCH_USE_CUDA_DSA'] = '1'
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # To find the line of code causing the error.

In [ ]:
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("2pcwkQgEjIHIdqgAHz1qpcsk4xB_7oY2An5GJKL3nZK958ZPg")

# Start ngrok tunnel, passing the port as an integer
public_url = ngrok.connect(8502)
print(f"Public URL: {public_url}")

!streamlit run app.py --server.port 8502 &

Public URL: NgrokTunnel: "https://5866-35-185-235-41.ngrok-free.app" -> "http://localhost:8502"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.185.235.41:8502

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2024-12-01 21:40:19.738861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 21:40:19.763724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 21:40:19.772631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 21:40:21.

In [1]:
!pip install huggingface_hub

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG